In [1]:
from Acesso import Login
from Query import Query
from Moeda import Moeda
import pandas as pd

pd.set_option('display.max_columns',None)

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

querys={

    'Meta':

    """

    SELECT * FROM netfeira.vw_metas
    WHERE [Meta R$]>0
    
    """,

    'Vendedor':

    """
    
    SELECT * FROM netfeira.vw_vendedor
    WHERE Categoria='CLT' AND [Status do Vendedor]='ATIVO' AND Telefone IS NOT NULL
    
    """,

    'Supervisor':

    """
    
    SELECT * FROM netfeira.vw_supervisor
    WHERE NOT Equipe LIKE '%120%'
    
    """,

    'Estatico':

    """
    
    DECLARE @DTBASE DATETIME, @DTFIM DATETIME,@DTINICIO DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SET @DTFIM=@DTBASE

    SET @DTINICIO=CONCAT(YEAR(@DTFIM),'-',MONTH(@DTFIM),'-01')

    SELECT * FROM netfeira.vw_targetestatico
    WHERE [Data de Faturamento] BETWEEN @DTINICIO AND @DTFIM AND [Tipo de Operação]='VENDAS'
    
    """,

    'Aberto':

    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SELECT *
	FROM netfeira.vw_aberto
    WHERE [Data do Pedido]=@DTBASE
    
    """,

    'Calendario':

    """
    
    DECLARE @DTBASE DATETIME

    SET @DTBASE=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101)

    SELECT COUNT(DATA) AS [Útil],COUNT([Data Trabalhada])-1 AS [Trabalhado],
    COUNT(DATA)-(COUNT([Data Trabalhada])-1) AS [Restante]
    FROM netfeira.vw_calend
    WHERE YEAR(Data)=YEAR(@DTBASE) AND MONTH(Data)=MONTH(@DTBASE) AND [Dia Útil]=1    
    
    """

}

In [2]:
df=sql.CriarTabela(kwargs=querys)

In [3]:
df['Estatico'].columns

Index(['ID Empresa', 'Pedido', 'Nfe', 'ID Cliente', 'ID Vendedor',
       'Data de Emissão', 'Data de Faturamento', 'ID Motivo', 'Situação',
       'Tipo de Pedido', 'Tipo de Operação', 'ID Roteiro', 'ID Usuário',
       'Tabelas', 'Origem', 'Tipo de Entrega', 'Seq Roteiro', 'SKU', 'Seq',
       'Qtde', 'Unid. VDA', 'Qtde VDA', 'Valor VDA', 'Total Venda', 'Total AV',
       'Total Geral', 'Margem Bruta R$', 'Comissão R$', 'IPI R$', 'PIS R$',
       'COFINS R$', 'ICMS R$', 'ICMS ST R$', 'Peso Bruto KG',
       'Peso Líquido KG', 'Cad Vendedor'],
      dtype='object')

In [4]:
df['Consolidado']=df['Estatico'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Consolidado'].rename(columns={'Total Venda':'Faturado'},inplace=True)

In [5]:
df['Pendente']=df['Aberto'].loc[df['Aberto']['Situação']=='AB'].groupby(['ID Vendedor'],as_index=False).agg({'Total Venda':'sum'})

df['Pendente'].rename(columns={'Total Venda':'Em Aberto'},inplace=True)

In [6]:
df['Meta']=df['Meta'].groupby(['ID Vendedor'],as_index=False).agg({'Meta R$':'sum'})

In [7]:
df['Meta']=df['Meta'].merge(df['Consolidado'],on='ID Vendedor',how='left')

df['Meta']=df['Meta'].merge(df['Pendente'],on='ID Vendedor',how='left')

In [8]:
colunas=df['Meta'].columns[1:]

for coluna in colunas:
    
    df['Meta'].loc[df['Meta'][coluna].isnull(),coluna]=0    
    
    pass

df['Meta']

,ID Vendedor,Meta R$,Faturado,Em Aberto
0,ADANTAS,299900.00,203552.00,5895.72
1,ADELIALO,141500.00,93520.43,3391.99
2,ADEMIRFE,58700.00,51796.19,1826.53
3,ADIVAL,10000.00,13697.29,0.00
4,ALENOGUE,6681.00,6601.07,0.00
5,ALERUIZ,65000.00,40124.61,6951.25
6,AVULSO,65000.00,21971.48,0.00
7,CELSOCAR,135600.00,91923.09,4895.69
8,CLAUDIOP,122400.00,96772.29,11508.83
9,CLISEMCO,2000.00,62.54,0.00


In [9]:
df['Meta']['Realizado R$']=round(df['Meta']['Faturado']+df['Meta']['Em Aberto'],2)

In [10]:
df['Meta']['Dif']=round(df['Meta']['Realizado R$']-df['Meta']['Meta R$'],2)

In [11]:
df['Meta']['Perc']=round(df['Meta']['Realizado R$']/df['Meta']['Meta R$'],4)*100

In [12]:
util=df['Calendario']['Útil'].max()

trabalhado=df['Calendario']['Trabalhado'].max()

In [13]:
df['Meta']['Projeção']=round((df['Meta']['Realizado R$']/trabalhado)*util,2)

In [14]:
df['Meta']['Perc Projeção']=round(df['Meta']['Projeção']/df['Meta']['Meta R$'],4)*100

In [15]:
colunas=df['Meta'].columns[1:]

for coluna in colunas:
    
    df['Meta'].loc[df['Meta'][coluna].isnull(),coluna]=0    
    
    pass

df['Meta']

,ID Vendedor,Meta R$,Faturado,Em Aberto,Realizado R$,Dif,Perc,Projeção,Perc Projeção
0,ADANTAS,299900.00,203552.00,5895.72,209447.72,-90452.28,69.84,299211.03,99.77
1,ADELIALO,141500.00,93520.43,3391.99,96912.42,-44587.58,68.49,138446.31,97.84
2,ADEMIRFE,58700.00,51796.19,1826.53,53622.72,-5077.28,91.35,76603.89,130.50
3,ADIVAL,10000.00,13697.29,0.00,13697.29,3697.29,136.97,19567.56,195.68
4,ALENOGUE,6681.00,6601.07,0.00,6601.07,-79.93,98.80,9430.10,141.15
5,ALERUIZ,65000.00,40124.61,6951.25,47075.86,-17924.14,72.42,67251.23,103.46
6,AVULSO,65000.00,21971.48,0.00,21971.48,-43028.52,33.80,31387.83,48.29
7,CELSOCAR,135600.00,91923.09,4895.69,96818.78,-38781.22,71.40,138312.54,102.00
8,CLAUDIOP,122400.00,96772.29,11508.83,108281.12,-14118.88,88.46,154687.31,126.38
9,CLISEMCO,2000.00,62.54,0.00,62.54,-1937.46,3.13,89.34,4.47


In [16]:
df['Vendedor'].columns

Index(['ID Vendedor', 'Vendedor', 'Nome Resumido', 'ID Equipe', 'E-mail',
       'Categoria', 'Data de Cadastro', 'Status do Vendedor', 'DDD',
       'Telefone'],
      dtype='object')

In [17]:
df['Supervisor'].columns

Index(['ID Equipe', 'Equipe', 'ID Sup', 'Supervisor', 'Email Sup', 'DDD Sup',
       'Telefone Sup', 'ID Gerente', 'Gerente', 'Email Gerente', 'DDD Gerente',
       'Telefone Gerente'],
      dtype='object')

In [18]:
df['Vendedor']=df['Vendedor'].merge(df['Supervisor'],on='ID Equipe',how='inner')[['ID Vendedor', 'Vendedor', 'Nome Resumido', 'Equipe','DDD',
       'Telefone']]

In [21]:
df['Vendedor']=df['Vendedor'].merge(df['Meta'],on='ID Vendedor',how='inner')